# Análisis de Estilo en Portafolios

<img style="float: right; margin: 0px 0px 15px 15px;" src="http://www.picpedia.org/finance/images/stock-portfolio.jpg" width="400px" height="400px" />

> Al analizar si invertir o no en un portafolio (fondo de inversión), nos podríamos preguntar qué tan bueno es el administrador de dicho fondo.

> Bien, esto nos lleva a preguntarnos cómo es que se conforma un portafolio, cuál es el proceso mediante el cual se seleccionan los activos.

> Se puede consultar que los portafolios se venden con ciertas tendencias claramente marcadas (portafolios de compañías de alta capitalización, portafolios de compañías de baja capitalización, portafolios de compañías con baja diferencia de valor en libros y valor bursátil, ...).

> Entonces, surgen dos preguntas:
> 1. ¿Presenta en realidad el portafolio un comportamiento acorde al estilo que se promete?
> 2. Si los activos que conforman el portafolio pertenecen a un grupo selecto, ¿qué tan bueno es el administrador escogiendo dichos activos?

**Objetivos**

- Descomponer el exceso de rendimientos de un portafolio, en los *rendimientos debidos al estilo* y los *rendimientos debidos a la selección*.

- *rendimiento debido a la selección* = resta entre rendimiento del portafolio, y una mezcla pasiva de estilos similares.

**Solo el rendimiento debido a la selección se le puede atribuir al talento del administrador.**
___

## 1. Fundamentos básicos

- El análisis de estilo es una técnica poderosa y sofisticada para medir qué tan bueno es un administrador de fondos.

- Describe qué tan bueno es un administrador, analizando estadísticamente cómo los rendimientos del portafolio actúan, contra los rendimientos de lo que incluye el portafolio.

- En realidad nos interesa poco qué activos individuales contiene el portafolio. Más bien, miramos cómo se relaciona a diferentes estilos de inversión.

- Pero, ¿qué es un estilo de inversión?
 - Se puede pensar como un índice (método de inversión pasiva), o un *benchmark* (inversión base para comparar).
 
- Podemos pensar como si partiéramos el mercado en una matriz tres por tres, basados en la capitalización y en la diferencia entre valor en libros y valor de mercado.

|                                |                                |                               |
| ------------------------------ | ------------------------------ | ----------------------------- |
| Alta capitalización, Alta BM   | Alta Capitalización, Medio BM  | Alta Capitalización, Bajo BM  |
| Media capitalización, Alta BM  | Media Capitalización, Medio BM | Media Capitalización, Bajo BM |
| Baja capitalización, Alta BM   | Baja Capitalización, Medio BM  | Baja Capitalización, Bajo BM  |

- Entonces, por ejemplo, un administrador de fondos que invierte mayormente en activos de baja capitalización, diríamos que sigue un *estilo de baja capitalización*.

- Cabe destacar que estos no son los únicos estilos de inversión: también hay *estilo de bonos de gobierno*, por ejemplo.

Entonces, el análisis de estilo puede ser usado en varias formas:

1. Demostrar que un fondo de inversión sigue el objetivo que se plantea (si es de baja capitalización, etc)

2. Para descubrir cuál es el verdadero estilo de inversión del portafolio.

3. Evaluar que tan bueno es el administrador del fondo seleccionando los activos adecuados dentro del estilo prometido (valuación).

## 2. ¿Cómo se hace?

- Para explorar el funcionamiento del análisis de estilo, analizaremos el fondo **Vanguard Small-Cap Index Fund Admiral Shares (VSMAX)**.

- También necesitaremos los índices que marcan los diferentes estilos de inversión. Para ello usaremos las esquinas RUSSELL:
 - Russell 1000 Growth (^RLG)
 - Russell 1000 Value (^RLV)
 - Russell 2000 Growth (^RUO)
 - Russell 2000 Value (^RUJ)

Descarguemos datos diarios en los últimos 5 años.

In [1]:
# Importamos librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import yfinance as yf

In [2]:
# Nombres de instrumentos que vamos a importar
tickers = ['VSMAX', '^RLG', '^RLV', '^RUO', '^RUJ']
# Fechas de inicio y fin
start_date = "2000-01-01"
# Descargamos datos
closes = yf.download(tickers, start=start_date)['Close']


[*********************100%%**********************]  5 of 5 completed


Obtenemos rendimientos

In [3]:
# Rendimientos diarios
rt = closes.pct_change().dropna()

El análisis de estilo no es más que una regresión lineal de los (excesos de) rendimientos del portafolio que se va a analizar (*variable dependiente*), respecto a los diferentes estilos de inversión (*variables independientes o explicativas*).

1. De forma que se asignan pesos (o ponderaciones) a cada uno de los estilos elegidos y una ponderación adicional como intercepto <font color=blue> (ver en el tablero)</font>.

 $$r_{p,t}=\alpha+w_1I_{1,t}+w_2I_{2,t}+\dots+w_nI_{n,t}+\epsilon_t,$$
 
 donde $\epsilon_t$ es el término de error (residual). Se incluye la restricción de que dichos pesos deben sumar uno (1), para interpretarlos como porcentajes:
 
 $$w_1+w_2+\dots+w_n=1.$$

2. Se calculan los residuales (error) al cuadrado en cada paso, y se minimiza el total: **mínimos cuadrados**.
 
 <font color=blue> (ver en el tablero)</font>.
 
 $$\min_{\alpha,w_1,\dots,w_n} \sum_{t=0}^T\epsilon_t^2=\sum_{t=0}^T\left(r_{p,t}-\alpha-w_1I_{1,t}-w_2I_{2,t}-\dots-w_nI_{n,t}\right)^2$$
 
 sujeto a $w_1+w_2+\dots+w_n=1.$

3. Calcular el estadístico $R$-cuadrado: en qué porcentaje la variación de los rendimientos de los índices explican la variación de los rendimientos del portafolio.

 $$R^2=1-\frac{var(\epsilon)}{var(r_p)}$$

Entonces, necesitamos un optimizador

In [4]:
# Importamos la librería optimize de scipy
from scipy.optimize import minimize

Debemos escribir la función a optimizar

In [5]:
def model_fit(X, y):
    def obj(coef, r_fondo, r_estilos):
        alpha = coef[0]
        w = coef[1:]
        modelo = alpha + r_estilos.dot(w)
        residuales = r_fondo - modelo
        return (residuales**2).mean()


    coef0 = [0, 0.25, 0.25, 0.25, 0.25]
    cons = {'type': 'eq', 'fun': lambda coef: coef[1:].sum() - 1}
    bnds = ((None, None),) + ((0, None),) * 4
    # Resolvemos
    style = minimize(
        fun=obj,
        x0=coef0,
        args=(y, X),
        bounds=bnds,
        constraints=cons,
        tol=1e-10
    )
    return style

Datos de entrada al optimizador

In [6]:
# Datos
y = rt['VSMAX']
X = rt[['^RLG', '^RLV', '^RUO', '^RUJ']]

In [7]:
# Entrena el modelo
model = model_fit(X, y)

In [8]:
# Coeficientes alpha
model.x

array([3.40342013e-05, 1.06849555e-01, 1.28067796e-01, 3.75207313e-01,
       3.89875336e-01])

In [9]:
# Pesos
model.x[0]

3.403420125205318e-05

### Si el peso de alpha es 0, quiere decir que basta con las betas para explicar la acción; lo cual es bueno porque tiene info que el mercado no tiene. 

In [10]:
# Verificamos pesos 


### ¿Qué podemos concluir?

 - Russell 1000 Growth (^RLG)
 - Russell 1000 Value (^RLV)
 - Russell 2000 Growth (^RUO)
 - Russell 2000 Value (^RUJ)

Entonces, el fondo **Vanguard Small-Cap Index Fund Admiral Shares (VSMAX)**, está conformado en un

### Estas son nuevas betas

In [11]:
# w_1
model.x[1] #el 10% esta en la primera que es cara y grande

0.10684955505923037

por activos de alta capialización y con baja diferencia BM (Russell 1000 Growth); está conformado en un

In [12]:
# w_2
model.x[2] #1 &2 son las grandes empresas 

0.12806779581781222

por activos de alta capitalización con alta diferencia BM (Russell 1000 Value); está conformado en un

In [13]:
# w_3
model.x[3] #para growth chicos en crecimiento

0.3752073126817489

por activos de baja capitalización con baja diferencia BM (Russell 2000 Growth); y está conformado en un

In [14]:
# w_4
model.x[4] #también es chica y barata, o sea tiene más peso en chicas que son las 3 & 4.

0.3898753364412086

por activos de baja capitalización con alta diferencia BM (Russell 2000 Value). El restante

In [15]:
# alpha
model.x[0]

3.403420125205318e-05

In [16]:
# Usa el modelo para predecir 
X.keys()

Index(['^RLG', '^RLV', '^RUO', '^RUJ'], dtype='object')

In [17]:
predicciones = model.x[1] * X['^RLG'] + model.x[2] * X['^RLV'] + model.x[3] * X['^RUO'] + model.x[4] * X['^RUJ']

In [18]:
predicciones

Date
2002-10-01    0.021125
2002-10-02   -0.022052
2002-10-03   -0.009978
2002-10-04   -0.024475
2002-10-07   -0.026012
                ...   
2023-11-08   -0.008440
2023-11-09   -0.013948
2023-11-10    0.011605
2023-11-13   -0.000082
2023-11-14    0.046509
Length: 5318, dtype: float64

de los rendimientos diarios, **es atribuible al administrador del fondo por la selección particular de los activos.**

Por tanto, **el estilo predominante del fondo es de baja capitalizacion** ($\approx 70\%$).

Calculamos el $R^2$

In [19]:
from sklearn.metrics import r2_score

In [20]:
r2_score(y_pred=predicciones, y_true=y)

0.9862685622530297

Estadísticamente, el modelo propuesto explica el $98.33\%$ de la variación total de los rendimientos del portafolio.

#### Formando un portafolio

- Checa con : Berkshiere, Apple, apple y amazon, apple, amazon y microsoft

¿qué concluyes?


In [21]:
# Nombres de instrumentos que vamos a importar
tickers = ['VSMAX', '^RLG', '^RLV', '^RUO', '^RUJ',
          'BRK-A', 'AAPL', 'AMZN', 'MSFT']
# Fechas de inicio y fin
start_date = "2000-01-01"
# Descargamos datos
closes = yf.download(tickers, start=start_date)['Close']

[*********************100%%**********************]  9 of 9 completed


In [22]:
rt = closes.pct_change().dropna()

In [23]:
y = rt['AAPL']
X = rt[['^RLG', '^RLV', '^RUO', '^RUJ']]

model_fit(X, y)
predicciones = model.x[1] * X['^RLG'] + model.x[2] * X['^RLV'] + model.x[3] * X['^RUO'] + model.x[4] * X['^RUJ']
r2_score(y_pred=predicciones, y_true=y)

0.3053985038555992

In [24]:
#su r2 es 30% aapl

In [25]:
y = rt['AMZN']
X = rt[['^RLG', '^RLV', '^RUO', '^RUJ']]

model_fit(X, y)
predicciones = model.x[1] * X['^RLG'] + model.x[2] * X['^RLV'] + model.x[3] * X['^RUO'] + model.x[4] * X['^RUJ']
r2_score(y_pred=predicciones, y_true=y)

0.26492120053575074

In [26]:
#su r2 es 26% de amzn

In [27]:
y = rt['BRK-A']
X = rt[['^RLG', '^RLV', '^RUO', '^RUJ']]

model_fit(X, y)
predicciones = model.x[1] * X['^RLG'] + model.x[2] * X['^RLV'] + model.x[3] * X['^RUO'] + model.x[4] * X['^RUJ']
r2_score(y_pred=predicciones, y_true=y)

0.14695682293418055

In [28]:
#su r2 es 14% de amzn

In [29]:
#1/3 apple, 1/3 msft, 1/3 amazon
portafolio = (1/3)* rt['AAPL'] + (1/3)* rt['MSFT'] + (1/3)* rt['AMZN'] 
y = portafolio
X = rt[['^RLG', '^RLV', '^RUO', '^RUJ']]

model_fit(X, y)
predicciones = model.x[1] * X['^RLG'] + model.x[2] * X['^RLV'] + model.x[3] * X['^RUO'] + model.x[4] * X['^RUJ']
r2_score(y_pred=predicciones, y_true=y)

0.467320095054674

In [30]:
model_fit(X, y).x

array([6.60929084e-04, 1.00000000e+00, 7.89170009e-17, 7.63278329e-17,
       0.00000000e+00])

In [31]:
#la primera es grande y cara, y casi todo se va ahí;la beta siempre sera 1 en empresas grandes y caras, en el resto serán 0.